In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import io
from PIL import Image

from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger, ReduceLROnPlateau
%load_ext tensorboard

import os
import matplotlib.pylab as plt
import numpy as np
import math
import datetime
import pandas as pd

print("Version: ", tf.__version__)
tf.get_logger().setLevel('INFO')

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Version:  2.6.0-dev20210331


In [2]:
# Download and prepare the horses or humans dataset

splits, info = tfds.load('horses_or_humans', as_supervised=True, with_info=True, split=['train[:80%]', 'train[80%:]', 'test'])

(train_examples, validation_examples, test_examples) = splits

num_examples = info.splits['train'].num_examples
num_classes = info.features['label'].num_classes

Dataset horses_or_humans downloaded and prepared to C:\Users\Gaya3selva\tensorflow_datasets\horses_or_humans\3.0.0. Subsequent calls will reuse this data.


In [3]:
SIZE = 150 #@param {type:"slider", min:64, max:300, step:1}
IMAGE_SIZE = (SIZE, SIZE)

In [4]:
def format_image(image, label):
  image = tf.image.resize(image, IMAGE_SIZE) / 255.0
  return  image, label

In [5]:
BATCH_SIZE = 32 #@param {type:"integer"}

In [7]:
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)
train_batches = train_examples.shuffle(num_examples // 4).map(format_image).batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_examples.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_batches = test_examples.map(format_image).batch(1)

In [8]:
for image_batch, label_batch in train_batches.take(1):
  pass

image_batch.shape

TensorShape([32, 150, 150, 3])

In [9]:
def build_model(dense_units, input_shape=IMAGE_SIZE + (3,)):
  model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(dense_units, activation='relu'),
      tf.keras.layers.Dense(2, activation='softmax')
  ])
  return model

In [10]:
!rm -rf logs

'rm' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

model.fit(train_batches, 
          epochs=10, 
          validation_data=validation_batches, 
          callbacks=[tensorboard_callback])

Epoch 1/10
26/26 [==============================] - 19s 635ms/step - loss: 0.6770 - accuracy: 0.5781 - val_loss: 0.6474 - val_accuracy: 0.6341
Epoch 2/10
26/26 [==============================] - 13s 482ms/step - loss: 0.6349 - accuracy: 0.6537 - val_loss: 0.6478 - val_accuracy: 0.5805
Epoch 3/10
26/26 [==============================] - 13s 484ms/step - loss: 0.6016 - accuracy: 0.6606 - val_loss: 0.7351 - val_accuracy: 0.5171
Epoch 4/10
26/26 [==============================] - 13s 479ms/step - loss: 0.6054 - accuracy: 0.6539 - val_loss: 0.5216 - val_accuracy: 0.7805
Epoch 5/10
26/26 [==============================] - 13s 487ms/step - loss: 0.5116 - accuracy: 0.7547 - val_loss: 0.5041 - val_accuracy: 0.7512
Epoch 6/10
26/26 [==============================] - 13s 508ms/step - loss: 0.4727 - accuracy: 0.7994 - val_loss: 0.4195 - val_accuracy: 0.8244
Epoch 7/10
26/26 [==============================] - 13s 502ms/step - loss: 0.4389 - accuracy: 0.7929 - val_loss: 0.4863 - val_accuracy: 0.7366

In [12]:
%tensorboard --logdir logs

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2021-04-12 18:52:51.434242: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-04-12 18:52:51.436357: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-04-12 18:52:51.504420: W .\tensorflow/core/platform/env.h:647] Using modular file system for 'gs. Please switch to tensorflow-io (https://github.com/tensorflow/io) for file system support of 'gs'.
2021-04-12 18:52:51.507277: W .\tensorflow/core/platform/env.h:647] Using modular file system for 's3. Please switch to tensorflow-io (https://github.com/tensorflow/io) for file system support of 's3'.
Traceback (most recent call last):
  File "c:\users\gaya3selva\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\gaya3selva\anaconda3\l

In [13]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('weights.{epoch:02d}-{val_loss:.2f}.h5', verbose=1),
          ])

Epoch 1/5
26/26 - 14s - loss: 0.6722 - accuracy: 0.5888 - val_loss: 0.6507 - val_accuracy: 0.6049

Epoch 00001: saving model to weights.01-0.65.h5
Epoch 2/5
26/26 - 12s - loss: 0.6343 - accuracy: 0.6740 - val_loss: 0.6867 - val_accuracy: 0.4829

Epoch 00002: saving model to weights.02-0.69.h5
Epoch 3/5
26/26 - 13s - loss: 0.5981 - accuracy: 0.7032 - val_loss: 0.5573 - val_accuracy: 0.8439

Epoch 00003: saving model to weights.03-0.56.h5
Epoch 4/5
26/26 - 13s - loss: 0.5265 - accuracy: 0.7774 - val_loss: 0.5919 - val_accuracy: 0.6195

Epoch 00004: saving model to weights.04-0.59.h5
Epoch 5/5
26/26 - 13s - loss: 0.4815 - accuracy: 0.7822 - val_loss: 0.6421 - val_accuracy: 0.5805

Epoch 00005: saving model to weights.05-0.64.h5


In [14]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=1, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('saved_model', verbose=1)
          ])

26/26 - 14s - loss: 0.6815 - accuracy: 0.5584 - val_loss: 0.6683 - val_accuracy: 0.5805

Epoch 00001: saving model to saved_model
INFO:tensorflow:Assets written to: saved_model\assets


INFO:tensorflow:Assets written to: saved_model\assets


In [15]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=2, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[ModelCheckpoint('model.h5', verbose=1)
          ])

Epoch 1/2
26/26 - 14s - loss: 0.6748 - accuracy: 0.5633 - val_loss: 0.6491 - val_accuracy: 0.7268

Epoch 00001: saving model to model.h5
Epoch 2/2
26/26 - 14s - loss: 0.6263 - accuracy: 0.7178 - val_loss: 0.6126 - val_accuracy: 0.7463

Epoch 00002: saving model to model.h5


In [16]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          verbose=2,
          callbacks=[EarlyStopping(
              patience=3,
              min_delta=0.05,
              baseline=0.8,
              mode='min',
              monitor='val_loss',
              restore_best_weights=True,
              verbose=1)
          ])

Epoch 1/50
26/26 - 14s - loss: 0.6739 - accuracy: 0.5487 - val_loss: 0.6804 - val_accuracy: 0.4341
Epoch 2/50
26/26 - 12s - loss: 0.6453 - accuracy: 0.6241 - val_loss: 0.6429 - val_accuracy: 0.6146
Epoch 3/50
26/26 - 13s - loss: 0.6027 - accuracy: 0.7311 - val_loss: 0.6415 - val_accuracy: 0.5366
Epoch 4/50
26/26 - 13s - loss: 0.5567 - accuracy: 0.7543 - val_loss: 0.5218 - val_accuracy: 0.7902
Epoch 5/50
26/26 - 13s - loss: 0.5078 - accuracy: 0.7762 - val_loss: 0.5626 - val_accuracy: 0.6634
Epoch 6/50
26/26 - 13s - loss: 0.4456 - accuracy: 0.8175 - val_loss: 0.4646 - val_accuracy: 0.7659
Epoch 7/50
26/26 - 12s - loss: 0.3827 - accuracy: 0.8601 - val_loss: 0.3968 - val_accuracy: 0.8585
Epoch 8/50
26/26 - 15s - loss: 0.3242 - accuracy: 0.8844 - val_loss: 0.2338 - val_accuracy: 0.9512
Epoch 9/50
26/26 - 12s - loss: 0.2385 - accuracy: 0.9355 - val_loss: 0.2706 - val_accuracy: 0.8976
Epoch 10/50
26/26 - 12s - loss: 0.1854 - accuracy: 0.9538 - val_loss: 0.1265 - val_accuracy: 0.9854
Epoch 11/

In [17]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
csv_file = 'training.csv'

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[CSVLogger(csv_file)
          ])

Epoch 1/5
26/26 [==============================] - 13s 473ms/step - loss: 0.6902 - accuracy: 0.5275 - val_loss: 0.7403 - val_accuracy: 0.4341
Epoch 2/5
26/26 [==============================] - 12s 449ms/step - loss: 0.6630 - accuracy: 0.5935 - val_loss: 0.6488 - val_accuracy: 0.6049
Epoch 3/5
26/26 [==============================] - 12s 452ms/step - loss: 0.6309 - accuracy: 0.6768 - val_loss: 0.6449 - val_accuracy: 0.5171
Epoch 4/5
26/26 [==============================] - 12s 450ms/step - loss: 0.5807 - accuracy: 0.7530 - val_loss: 0.5309 - val_accuracy: 0.8488
Epoch 5/5
26/26 [==============================] - 12s 465ms/step - loss: 0.5372 - accuracy: 0.7735 - val_loss: 0.5207 - val_accuracy: 0.7268


In [18]:
pd.read_csv(csv_file).head()

,epoch,accuracy,loss,val_accuracy,val_loss
0,0,0.542579,0.679912,0.434146,0.740302
1,1,0.630170,0.655083,0.604878,0.648838
2,2,0.693431,0.624134,0.517073,0.644863
3,3,0.784672,0.566057,0.848781,0.530865
4,4,0.778589,0.519325,0.726829,0.520663


In [19]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
def step_decay(epoch):
	initial_lr = 0.01
	drop = 0.5
	epochs_drop = 1
	lr = initial_lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
	return lr

model.fit(train_batches, 
          epochs=5, 
          validation_data=validation_batches, 
          callbacks=[LearningRateScheduler(step_decay, verbose=1),
                    TensorBoard(log_dir='./log_dir')])

Epoch 1/5

Epoch 00001: LearningRateScheduler reducing learning rate to 0.005.
26/26 [==============================] - 16s 603ms/step - loss: 0.6863 - accuracy: 0.4975 - val_loss: 0.6859 - val_accuracy: 0.4390
Epoch 2/5

Epoch 00002: LearningRateScheduler reducing learning rate to 0.0025.
26/26 [==============================] - 12s 453ms/step - loss: 0.6583 - accuracy: 0.5535 - val_loss: 0.6695 - val_accuracy: 0.4732
Epoch 3/5

Epoch 00003: LearningRateScheduler reducing learning rate to 0.00125.
26/26 [==============================] - 12s 450ms/step - loss: 0.6548 - accuracy: 0.5939 - val_loss: 0.6660 - val_accuracy: 0.4878
Epoch 4/5

Epoch 00004: LearningRateScheduler reducing learning rate to 0.000625.
26/26 [==============================] - 12s 469ms/step - loss: 0.6409 - accuracy: 0.6038 - val_loss: 0.6592 - val_accuracy: 0.5415
Epoch 5/5

Epoch 00005: LearningRateScheduler reducing learning rate to 0.0003125.
26/26 [==============================] - 12s 460ms/step - loss: 0.6

In [20]:
%tensorboard --logdir log_dir

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
2021-04-12 19:11:01.506100: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2021-04-12 19:11:01.508363: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2021-04-12 19:11:01.574926: W .\tensorflow/core/platform/env.h:647] Using modular file system for 'gs. Please switch to tensorflow-io (https://github.com/tensorflow/io) for file system support of 'gs'.
2021-04-12 19:11:01.577470: W .\tensorflow/core/platform/env.h:647] Using modular file system for 's3. Please switch to tensorflow-io (https://github.com/tensorflow/io) for file system support of 's3'.
Traceback (most recent call last):
  File "c:\users\gaya3selva\anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\gaya3selva\anaconda3\l

In [21]:
model = build_model(dense_units=256)
model.compile(
    optimizer='sgd',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])
  
model.fit(train_batches, 
          epochs=50, 
          validation_data=validation_batches, 
          callbacks=[ReduceLROnPlateau(monitor='val_loss', 
                                       factor=0.2, verbose=1,
                                       patience=1, min_lr=0.001),
                     TensorBoard(log_dir='./log_dir')])

Epoch 1/50
26/26 [==============================] - 15s 544ms/step - loss: 0.6882 - accuracy: 0.5103 - val_loss: 0.6333 - val_accuracy: 0.6049
Epoch 2/50
26/26 [==============================] - 12s 469ms/step - loss: 0.6201 - accuracy: 0.6964 - val_loss: 0.5533 - val_accuracy: 0.8439
Epoch 3/50
26/26 [==============================] - 12s 456ms/step - loss: 0.5462 - accuracy: 0.7826 - val_loss: 0.4832 - val_accuracy: 0.8537
Epoch 4/50
26/26 [==============================] - 12s 469ms/step - loss: 0.4906 - accuracy: 0.7816 - val_loss: 0.3983 - val_accuracy: 0.8732
Epoch 5/50
26/26 [==============================] - 12s 471ms/step - loss: 0.4422 - accuracy: 0.7989 - val_loss: 0.3166 - val_accuracy: 0.9024
Epoch 6/50
26/26 [==============================] - 12s 471ms/step - loss: 0.3186 - accuracy: 0.9000 - val_loss: 0.3056 - val_accuracy: 0.8829
Epoch 7/50
26/26 [==============================] - 12s 477ms/step - loss: 0.2730 - accuracy: 0.9158 - val_loss: 0.1988 - val_accuracy: 0.9659

In [ ]:
%tensorboard --logdir log_dir